In [4]:
import os
import numpy as np


def unpack(buf,dtype,count,offset):
    # Interpret a buffer as a 1-dimensional array.
    data=np.frombuffer(buf,dtype,count,offset)
    return data[0] if count==1 else data

def IQ_unpack(data):
    data_=np.empty_like(data,dtype='float32')
    
    for i,v in enumerate(data):
        E=v>>12
        S=(v&0x0FFF)>>11
        M=v&0x07FF
        SM=v&0x0FFF
        
        if E!=0:
            M1=np.array(M,dtype='int32')
            if S:
                M1|=0x800
            else:
                M1|=0xfffff000
                
            E=np.array(E,dtype='float32')
            data_[i]=M1*2**(E-25)
        else:
            data_[i]=SM*2**(-24)
    
    return data_[0::2],data_[1::2]

def read(path):
    '''load into buffer'''
    file=open(path,'rb')
    buf=file.read()
    file.close()
    
    data={}
    
    '''header'''
    data['h']={}
    data['h']['version']=unpack(buf,'uint8',1,0)
    data['h']['sitename']=unpack(buf,'S16',1,1)
    data['h']['wavelength']=unpack(buf,'float32',1,17)
    data['h']['vcp']=unpack(buf,'uint8',1,21)
    data['h']['pol']=unpack(buf,'uint8',1,22)
    print(data['h']['pol'])
    data['h']['pulsewidth']=unpack(buf,'float32',1,23)
    data['h']['calibration']=unpack(buf,'float32',1,27)
    data['h']['noise']=unpack(buf,'float32',1,31)
    data['h']['freq']=unpack(buf,'float32',1,35)
    data['h']['firstbin']=unpack(buf,'uint16',1,39)
    data['h']['phasecode']=unpack(buf,'uint8',1,41)
    data['h']['vnoise']=unpack(buf,'float32',1,42)
    data['h']['vcalib']=unpack(buf,'float32',1,46)
    
    ptr=128+256 #TSHeader128字节，保留字段256字节，TS估计是体扫的拼音
    
    '''data'''
    data['d']=[]
    while True:
        data['d'].append({})
        
        data['d'][-1]['time']=unpack(buf,'uint8',8,ptr)
        data['d'][-1]['clock']=unpack(buf,'uint32',1,ptr+8)
        data['d'][-1]['seqnum']=unpack(buf,'uint32',1,ptr+12)      
        data['d'][-1]['latitude']=unpack(buf,'uint32',1,ptr+16).astype('float32')/1000
        print('lat:',data['d'][-1]['latitude'])
        data['d'][-1]['longitude']=unpack(buf,'uint32',1,ptr+20).astype('float32')/1000
        data['d'][-1]['height']=unpack(buf,'uint32',1,ptr+24)
        data['d'][-1]['az']=unpack(buf,'uint16',1,ptr+28).astype('float32')/100
        print(data['d'][-1]['az'])
        data['d'][-1]['el']=unpack(buf,'uint16',1,ptr+30).astype('float32')/100
        data['d'][-1]['prf']=unpack(buf,'uint16',1,ptr+32)
        data['d'][-1]['samples']=unpack(buf,'uint16',1,ptr+34)
        data['d'][-1]['binnum']=unpack(buf,'uint16',1,ptr+36)
        print("binnum",data['d'][-1]['binnum'])
        data['d'][-1]['reso']=unpack(buf,'uint16',1,ptr+38)
        print("reso: %d"%data['d'][-1]['reso'])
        data['d'][-1]['mode']=unpack(buf,'uint8',1,ptr+40)
        
        data['d'][-1]['state']=unpack(buf,'uint32',1,ptr+41)  
        data['d'][-1]['sploblank']=unpack(buf,'uint8',1,ptr+45)
        data['d'][-1]['nextprf']=unpack(buf,'uint16',1,ptr+46)
        data['d'][-1]['burstmag']=unpack(buf,'float32',1,48)
        data['d'][-1]['burstang']=unpack(buf,'float32',1,52)
        data['d'][-1]['swpidx']=unpack(buf,'uint16',1,ptr+56)
        data['d'][-1]['anglereso']=unpack(buf,'uint16',1,ptr+58)
        data['d'][-1]['chan']=unpack(buf,'uint8',1,ptr+60)
       
        data['d'][-1]['length']=unpack(buf,'uint16',1,ptr+61)
        data['d'][-1]['burstbinnum']=unpack(buf,'uint16',1,ptr+63)
        
        data['d'][-1]['H_IQ']=unpack(buf,'uint16',data['d'][-1]['binnum']*2,ptr+128)
        if data['d'][-1]['chan']==2:
            data['d'][-1]['V_IQ']=unpack(buf,'uint16',data['d'][-1]['binnum']*2,ptr+128+data['d'][-1]['binnum']*2*2)
        data['d'][-1]['burst']=unpack(buf,'uint16',data['d'][-1]['burstbinnum']*2,ptr+128+data['d'][-1]['binnum']*2*2*2)
        
        ptr+=(128+(data['d'][-1]['binnum']*data['d'][-1]['chan']+data['d'][-1]['burstbinnum'])*2*2)
    
        if data['d'][-1]['state']==2 or data['d'][-1]['state']==4:
            break
    
    
    
        if len(data['d'])==133:
            break
        
    for i,v in enumerate(data['d']):
        H_I,H_Q=IQ_unpack(v['H_IQ'])
        #V_I,V_Q=IQ_unpack(v['V_IQ'])
        del data['d'][i]['H_IQ']
        #del data['d'][i]['V_IQ']
        data['d'][i]['H_IQ']=H_I+1j*H_Q
        #data['d'][i]['V_IQ']=V_I+1j*V_Q
        
    return data

In [6]:
path=r'Z9002_20220624_005007_10_CDX.IQ'
data=read(path)

3
lat: 0.0
0.0
binnum 0
reso: 0
lat: 0.0
0.0
binnum 0
reso: 0
lat: 0.0
180.69
binnum 496
reso: 0
lat: 0.0
180.71
binnum 496
reso: 0
lat: 0.0
180.73
binnum 496
reso: 0
lat: 0.0
180.74
binnum 496
reso: 0
lat: 0.0
180.76
binnum 496
reso: 0
lat: 0.0
180.77
binnum 496
reso: 0
lat: 0.0
180.79
binnum 496
reso: 0
lat: 0.0
180.8
binnum 496
reso: 0
lat: 0.0
180.82
binnum 496
reso: 0
lat: 0.0
180.83
binnum 496
reso: 0
lat: 0.0
180.85
binnum 496
reso: 0
lat: 0.0
180.86
binnum 496
reso: 0
lat: 0.0
180.88
binnum 496
reso: 0
lat: 0.0
180.9
binnum 496
reso: 0
lat: 0.0
180.91
binnum 496
reso: 0
lat: 0.0
180.93
binnum 496
reso: 0
lat: 0.0
180.94
binnum 496
reso: 0
lat: 0.0
180.96
binnum 496
reso: 0
lat: 0.0
180.97
binnum 496
reso: 0
lat: 0.0
180.99
binnum 496
reso: 0
lat: 0.0
181.0
binnum 496
reso: 0
lat: 0.0
181.02
binnum 496
reso: 0
lat: 0.0
181.03
binnum 496
reso: 0
lat: 0.0
181.05
binnum 496
reso: 0
lat: 0.0
181.06
binnum 496
reso: 0
lat: 0.0
181.08
binnum 496
reso: 0
lat: 0.0
181.1
binnum 496
reso: